## Analysis on the documents

In [5]:
import os 
import json
# generally checking for ocr errors
extracted_output_path = "/Users/ajay/Documents/Oncampus/TrainingFramework/extracted_metadata/metadata_outputs"
g325a = list()
certnat = list()
ocrfail = list()
def read_json(path):
    with open(os.path.join(extracted_output_path,path), 'r') as json_file:
        # Load the data from the file
        data = json.load(json_file)
    return data

for path in os.listdir(extracted_output_path):
    
    data = read_json(path)
    if data.get('is_cert_naturalization',None):
        certnat.append([int(path.split("_")[-1]),path])
    if data.get('is_g325a',None):
        g325a.append([int(path.split("_")[-1]),path])

    if data.get('is_cert_naturalization',None) is None:
        ocrfail.append([int(path.split("_")[-1]),path])

len(g325a),len(certnat), len(ocrfail)

(565, 118, 0)

In [7]:
import pandas as pd 

df = pd.read_csv("/Users/ajay/Documents/Oncampus/TrainingFramework/combined_csv.csv")
df.head()

,Afile,Page,Is_Redacted,url,Detected Text,Ocr_Dump_path,idx
0,A10001331,0,False,https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/m...,For citation purposes: Records Copied at the N...,/Users/ajay/Documents/Oncampus/TrainingFramewo...,0
1,A10001331,1,False,https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/m...,Name (Last In CAPS) First Middle File No. 150 ...,/Users/ajay/Documents/Oncampus/TrainingFramewo...,1
2,A10001331,2,False,https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/m...,SERVED SERVICE Pigst 8.48 MAR-2,/Users/ajay/Documents/Oncampus/TrainingFramewo...,2
3,A10001331,3,False,https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/m...,Copied at the National Archives at Kansas City...,/Users/ajay/Documents/Oncampus/TrainingFramewo...,3
4,A10001331,4,False,https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/m...,UNITED STATES DEPARTMENT OF JUSTICE Form appro...,/Users/ajay/Documents/Oncampus/TrainingFramewo...,4


In [8]:
dffail = df.iloc[[x[0] for x in ocrfail],:]

In [9]:
dffail

,Afile,Page,Is_Redacted,url,Detected Text,Ocr_Dump_path,idx


### Add url and document ID, add raw ocr text

In [ ]:
import os 
import json
import pandas as pd 

# self explanatory 
def read_json(path):
    with open(os.path.join(extracted_output_path,path), 'r') as json_file:
        # Load the data from the file
        data = json.load(json_file)
    return data

def get_doc_data(path):
    pathsplit = path.split("_")

    dev_idx = int(pathsplit[0])
    # if 'redacted' in path:
    #     newname = '_'.join([pathsplit[1],pathsplit[3]])
        
    # else:
    newname = '_'.join(pathsplit[1:])

    afile = newname.split("_")[0]
    pno = int(pathsplit[-1].split(".")[0])
    return newname, afile, dev_idx, pno

def write_text_file(path,text):
    try:
        with open(path, 'w') as file:
            file.write(text)
    except Exception as e:
        print("An error occurred:", str(e),path,text)


''' 
Things to extract:
    url
    doc_id
    afile_number
    ocr_text 
        get the text
        store it in a text file in a separate folder
        store the path (relative path) as a key
'''

extracted_output_path = "/Users/ajay/Documents/Oncampus/TrainingFramework/extracted_metadata/metadata_outputs"
ocr_output_folder = "/Users/ajay/Documents/Oncampus/TrainingFramework/extracted_metadata/text_ocr"
df = pd.read_csv("/Users/ajay/Documents/Oncampus/TrainingFramework/combined_csv.csv")

for path in os.listdir(extracted_output_path):
    data = read_json(path)

    newname, afile, dev_idx, pno = get_doc_data(path)


    data['doc_id'] = newname.split(".")[0]
    data['afile_number'] = afile
    data['dev_idx'] = dev_idx
    data['pagenumber'] = pno
    data['url'] = df.iloc[dev_idx]['url']
    data['is_afile_redacted'] = False
    data['is_afile_withdrawn'] = False
    ocr_text = df.iloc[dev_idx]['Detected Text']
    data['ocr_path'] = os.path.join("text_ocr",newname.split(".")[0]+".txt")

    if isinstance(ocr_text,float):
        ocr_text = "" # to handle nans
    
    pathsplit = path.split("_")
    if len(pathsplit) >= 4:
        if 'redact' in pathsplit[2]:
            data['is_afile_redacted'] = True
        elif 'withdr' in pathsplit[2]:
            data['is_afile_withdrawn'] = True

    
    

    old_name = os.path.join(extracted_output_path, path)
    new_name = os.path.join(extracted_output_path, newname)
    os.rename(old_name, new_name)
    write_text_file(os.path.join(ocr_output_folder, newname.split(".")[0]+".txt"),ocr_text)

    with open(new_name, "w") as json_file:
        json.dump(data, json_file)


In [ ]:
extracted_output_path = "/Users/ajay/Documents/Oncampus/TrainingFramework/extracted_metadata/metadata_outputs"
s = set()
for path in os.listdir(extracted_output_path):
    pathsplit = path.split("_")
    if len(pathsplit) >= 4:
        s.add(pathsplit[2])

print(s)

{'withdrawal', 'redacted', 'redacted(1)'}


## Final Changes to metadata

### Run Certnat (Update it with newer metadata)

In [2]:
import os 
os.environ['OPENAI_API_KEY'] = '########'

In [3]:
import json 
import torch
from tqdm import tqdm
import sys
module_path = "/Users/ajay/Documents/Oncampus/Deployment_Framework/"
sys.path.append(module_path)
from metadata_extraction.run_llm_pipeline import Extraction
from classification_models.image.load_classifier import Pretrained_Image_Classifier
from misc_techniques import detect_cert_nat
from misc_techniques.detect_G325A import detect_g325av5 
from misc_techniques import flair_extract
from metadata_extraction.run_llm_pipeline import Extraction
from classification_models.image.load_classifier import Pretrained_Image_Classifier
import pandas as pd 
# import os




In [4]:
pathtojson = "/Users/ajay/Documents/Oncampus/TrainingFramework/extracted_metadata/metadata_outputs"


In [5]:
def read_json(path):
    with open(os.path.join(pathtojson,path), 'r') as json_file:
        # Load the data from the file
        data = json.load(json_file)
    return data

def read_text(path):
    with open(os.path.join("/Users/ajay/Documents/Oncampus/TrainingFramework/extracted_metadata",path), 'r') as file:
        # Read the entire contents of the file
        contents = file.read()
    return contents

In [6]:
for path in os.listdir(pathtojson):
    data = read_json(path)
    if data.get('is_cert_naturalization',False):
        certnat = data

In [7]:
os.listdir(pathtojson).index(path)

25347

In [ ]:
ext_class = Extraction("./metadata_extraction/configs/fewshot_certnat.cfg", "./metadata_extraction/configs/examples_certnat.yml")
for path in os.listdir(pathtojson)[20235:]:
    data = read_json(path)
    if "is_g325a" not in data:
        data['is_g325a'] = False 
    if "is_cert_naturalization" not in data:
        data['is_cert_naturalization'] = False
    if "g325a_attributes" not in data:
        data['g325a_attributes'] = {}
    if "cert_naturalization_attributes" not in data:
        data['cert_naturalization_attributes'] = {}
    if data.get('is_cert_naturalization',False):
        # g325adata = data
        text = read_text(data['ocr_path'])
        output = ext_class.extract_metadata(f"""{text}""")
        metadata = {}
        for k,v in output.items():
            metadata[f"LLM_{k}"] = v 
        data['cert_naturalization_attributes'] = {} 
        data['cert_naturalization_attributes'] = metadata

    json.dump(data, open(os.path.join(pathtojson,path), 'w'), indent=4)
    


In [ ]:
metadata = {}
for k,v in output.items():
    metadata[f"LLM_{k}"] = v 
g325adata['cert_naturalization_attributes'] = {} 
g325adata['cert_naturalization_attributes'] = metadata

In [ ]:
g325adata

{'document_type': 'letter',
 'is_cert_naturalization': True,
 'g325a_attributes': {},
 'is_g325a': False,
 'countries': ['United States', 'Cuba'],
 'years': ['1970', '1903'],
 'doc_id': 'A12495368_0008',
 'afile_number': 'A12495368',
 'dev_idx': 2665,
 'pagenumber': 8,
 'url': 'https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/migrants-state_A12495368_0008/full/full/0/default.jpg',
 'is_afile_redacted': False,
 'is_afile_withdrawn': False,
 'ocr_path': 'text_ocr/A12495368_0008.txt',
 'cert_naturalization_attributes': {'LLM_NUMBER': ['9264028'],
  'LLM_DATE_OF_BIRTH': ['December 26', '1903'],
  'LLM_SEX': ['female'],
  'LLM_COMPLEXION': ['fair'],
  'LLM_COLOR_OF_EYES': ['brown'],
  'LLM_WEIGHT': ['170 pounds'],
  'LLM_VISIBILE_DISTINCTIVE_MARKS': ['none'],
  'LLM_MARITAL_STATUS': ['married'],
  'LLM_FORMER_NATIONALITY': ['Cuba'],
  'LLM_NATURALIZATION_COUNTRY': ['UNITED STATES', 'United States'],
  'LLM_NATURALIZATION_MUNICIPALITY': ['Miami'],
  'LLM_NATURALIZATION_STATE': ['Florida'],
  'LL

In [ ]:
ext_class = Extraction("./metadata_extraction/configs/fewshot_certnat.cfg", "./metadata_extraction/configs/examples_certnat.yml")

for path in os.listdir(pathtojson):
    data = read_json(path)
    if data.get('is_cert_naturalization',False):
        g325adata = data
        text = read_text(g325adata['ocr_path'])
        output = ext_class.extract_metadata(f"""{text}""")
        print("----OLD--------")
        print(data['url'])
        print(data)
        print("----NEW--------")
        print(output)
        break

----OLD--------
https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/migrants-state_A12495368_0008/full/full/0/default.jpg
{'document_type': 'letter', 'is_cert_naturalization': True, 'g325a_attributes': {}, 'is_g325a': False, 'countries': ['United States', 'Cuba'], 'years': ['1970', '1903'], 'doc_id': 'A12495368_0008', 'afile_number': 'A12495368', 'dev_idx': 2665, 'pagenumber': 8, 'url': 'https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/migrants-state_A12495368_0008/full/full/0/default.jpg', 'is_afile_redacted': False, 'is_afile_withdrawn': False, 'ocr_path': 'text_ocr/A12495368_0008.txt'}
----NEW--------
{'NUMBER': ['9264028'], 'DATE_OF_BIRTH': ['December 26', '1903'], 'SEX': ['female'], 'COMPLEXION': ['fair'], 'COLOR_OF_EYES': ['brown'], 'WEIGHT': ['170 pounds'], 'VISIBILE_DISTINCTIVE_MARKS': ['none'], 'MARITAL_STATUS': ['married'], 'FORMER_NATIONALITY': ['Cuba'], 'NATURALIZATION_COUNTRY': ['UNITED STATES', 'United States'], 'NATURALIZATION_MUNICIPALITY': ['Miami'], 'NATURALIZATION_STATE': ['Flo

In [ ]:
text

"CA TO BE FORWARDED TO No. 9264028 IMMIGRATION AND INATURALIZATION SERVICE 1 Petition No. 46,641 DUPLICATE AlienRegistration No. A-12 495 368 Personal,description of holder's of date of naturalization: Date of birth December 26, 1903 sex female ; complexion fair coloroferies brown color of hair brown height 5 feel 7 inches; weight 170 pounds; visible distinctive marks none Maritalstatus married Country of formernationality Cuba I certify that the description above given is brive, and ephotograph.offixed, hereto is, a likeness of me < Haux Hn Cumll ( Complete and true signature of holder) THE UNITED STATES OF AMERICA SOUTHERN DI STRICT OF FLORI DA } SS: Beit known, that at a termofthe District the United States Court of held/pursuantio lawal Miami, Florida on February 3, 1970 the Court shavingfoundshall MARIA ANA CUNILL then residing at Miami, Florida inlends loreside permanentlyin, the United. States when sorequired, by the naturalization Laws of he United States), had in all thereresp

In [ ]:
df[df['url']=='https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/migrants-state_A10009114_0000/full/full/0/default.jpg'].iloc[0]['Detected Text']

'THE TO BE FORWARDED TO No. 8806431 IMMIGRATION AND INATURALIZATION SERVICE Petition No. 65010 DUPLICATE Personal description, of holderas, of date of naturalization: Date of birth January 25, 1909 sex female ; complexion medium coloroleyes brown color of hair grey height 5 feet 4 inches; weight 117 pounds; visibledistincine marks none Maritalstatus widowed Country of formernationality Hungary I certify that the description, above given is true, and that,She ephotographs affixed, hereto, is a likeness of me. Room Toth ( Complete and true signature of holder) UNITED STATES OF AMERICA } SS: WESTERN DISTRICT OF NEW YORK Beit known, that at a termolthe District Court,of The United States, Western District of New York held pursuantlo lawal Buffalo, N. Y. on May 6th, 1965 the Court havingfoundshal ROSA TOTA then residing at 2148 Main Street, Buffalo, New York - 14214 inlends loreside permanently in the United. States when sorequired by the Rosa Toth Naturalization Laws ofthe United States), 

### G325A

Adding new keys to the files

In [ ]:
''' 
read the json
save extracted ouptut to new json and then compare both (do old, new)
'''

' \nread the json\nsave extracted ouptut to new json and then compare both (do old, new)\n'

In [ ]:
data,path

({'document_type': 'misc',
  'doc_id': 'A18029489_0048',
  'afile_number': 'A18029489',
  'dev_idx': 6486,
  'pagenumber': 48,
  'url': 'https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/migrants-state_A18029489_0048/full/full/0/default.jpg',
  'is_afile_redacted': False,
  'is_afile_withdrawn': False,
  'ocr_path': 'text_ocr/A18029489_0048.txt'},
 'A18029489_0048.json')

In [ ]:
for path in os.listdir(pathtojson):
    data = read_json(path)
    if data.get('is_g325a',False):
        g325adata = data

In [ ]:
g325adata

{'document_type': 'form',
 'is_cert_naturalization': False,
 'g325a_attributes': {'LLM_FORM_NUMBER': ['G-325A'],
  'LLM_REVISION_DATE': ['6/7/2023'],
  'LLM_OMB_NUMBER': ['1115-0066'],
  'LLM_COUNTRY': ['Honduras', 'USA'],
  'LLM_OCCUPATION': ['NONE', 'none'],
  'LLM_STATE_CITY': ['San Pedro Sula', 'Newburgh N.Y.', 'Cortez']},
 'is_g325a': True,
 'countries': ['United States', 'Honduras'],
 'years': ['1988', '1907', '1971', '2023'],
 'doc_id': 'A29089715_redacted_0046',
 'afile_number': 'A29089715',
 'dev_idx': 13573,
 'pagenumber': 46,
 'url': 'https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/migrants-state_A29089715_redacted_0046/full/full/0/default.jpg',
 'is_afile_redacted': True,
 'is_afile_withdrawn': False,
 'ocr_path': 'text_ocr/A29089715_redacted_0046.txt'}

In [ ]:
text = read_text(g325adata['ocr_path'])

In [ ]:
url = "https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/migrants-state_A70022932_redacted_0020/full/full/0/default.jpg" 
df = pd.read_csv("/Users/ajay/Documents/Oncampus/TrainingFramework/combined_csv.csv")
df[df['url']==url].iloc[0]['Detected Text']

"Screened bv NARA 7/28/2023 INSTRUCTIONS. USE TYPEWRITER. BE SURE ALL COPIES ARE LEGIBLE. Failure to answer fully all questions delays action. 04255 Do Not Remove Carbons: If type er is not available, print heavily in block letters n ball-point pen. *U.S. GPO: 1986-155-463 For sale by the Superintendent of Documents. U.S. Government Printing Office. Washington. D.C. 20402 (Per 100) U.S. Department of Justice FORM G-325A OMB No. 1115-0066 Immigration and Naturalization Service BIOGRAPHIC INFORMATION Approval expires 4-30-85 (Family name) (First name) (Middle name) MALE BIRTHDATE (Mo.-Day-Yr.) NATIONALITY FILE NUMBER NOLASCO-Vda. Sanchez, ISABEL FEMALE 06-17-12 Salvadorean A- ALL OTHER NAMES USED (Including names by previous marriages) CITY AND COUNTRY OF BIRTH SOCIAL SECURITY NO. Isabel Nolasco San Salvador, E1 Salvador (If any) FAMILY NAME FIRST NAME DATE, CITY AND COUNTRY OF BIRTH (If known) CITY AND COUNTRY OF RESIDENCE FATHER Nolasco, Prudencio Unkn. E1 Salvador Deceased MOTHER (Mai

In [ ]:
ext_class = Extraction("./metadata_extraction/configs/fewshot_v2.cfg", "./metadata_extraction/configs/examples_g325a_2.yml")


In [ ]:
for path in os.listdir(pathtojson):
    data = read_json(path)
    if data.get('is_g325a',False):
        g325adata = data
        output = ext_class.extract_metadata(f"""{text}""")

In [ ]:
for path in os.listdir(pathtojson):
    data = read_json(path)
    if data.get('is_g325a',False):
        g325adata = data
        text = read_text(g325adata['ocr_path'])
        output = ext_class.extract_metadata(f"""{text}""")
        print("----OLD--------")
        print(g325adata['url'])
        print(g325adata["g325a_attributes"])
        print("----NEW--------")
        print(output)
        break

----OLD--------
https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/migrants-state_A21130830_0020/full/full/0/default.jpg
{'LLM_FORM_NUMBER': ['G-325A'], 'LLM_REVISION_DATE': ['REV. 10-1-74'], 'LLM_OMB_NUMBER': ['43-R436'], 'LLM_NATIONALITY': ['Cuban'], 'LLM_STATE_CITY': ['N.Y.', 'Miami Fla', 'Hav Cuba', 'Miami Beach Fla'], 'LLM_EMPLOYER_ADDRESS': ['2001 CollinsAve Room 405'], 'LLM_COUNTRY': ['USA'], 'LLM_EMPLOYER': ['Biscaya'], 'LLM_OCCUPATION': ['Retiremant Residence']}
----NEW--------
{'FORM_NUMBER': ['G-325A'], 'REVISION_DATE': ['REV. 10-1-74'], 'OMB_NUMBER': ['43-R436'], 'NATIONALITY': ['Cuban'], 'PARENT_STATE_CITY': ['Hav Cuba', 'N.Y.'], 'MARRIAGE_DATE': ['2/7/07'], 'MARRIAGE_TERMINATION_DATE': ['1/12/73'], 'STATE_CITY': ['Miami Beach', 'Fla USA', 'Miami', 'Fla'], 'EMPLOYER': ['Biscaya'], 'OCCUPATION': ['Retiremant Residence']}


In [ ]:
text = df[df['url']=='https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/migrants-state_A20550522_redacted_0154/full/full/0/default.jpg'].iloc[0]['Detected Text']
output = ext_class.extract_metadata(f"""{text}""")
output

{'FORM_NUMBER': ['G-325A'],
 'REVISION_DATE': ['REV. 8-27-'],
 'OMB_NUMBER': ['43-R436'],
 'NATIONALITY': ['Indian'],
 'PARENT_STATE_CITY': ['Hydrebad', 'Sindh'],
 'EMPLOYER': ['None', 'NONE', 'none'],
 'PARENT_COUNTRY': ['Pakistan'],
 'MARRIAGE_DATE': ['2-28-40'],
 'STATE_CITY': ['L. A.', 'Poona Maharastra India'],
 'OCCUPATION': ['Self Employed']}

In [ ]:
text = df[df['url']=='https://d1b7k5w7yjwpfg.cloudfront.net/iiif/2/migrants-state_A20550522_redacted_0154/full/full/0/default.jpg'].iloc[0]['Detected Text']
output = ext_class.extract_metadata(f"""{text}""")
output

{'FORM_NUMBER': ['G-325A'],
 'REVISION_DATE': ['REV. 8-27-'],
 'OMB_NUMBER': ['43-R436'],
 'NATIONALITY': ['Indian'],
 'STATE_CITY': ['Hydrebad', 'Poona Maharastra', 'Sindh', 'L. A.'],
 'EMPLOYER': ['None', 'NONE', 'none'],
 'COUNTRY': ['India', 'Pakistan'],
 'OCCUPATION': ['Self Employed']}

Not using new metadata schema for G325A because of lesser accuracy